In [1]:
%load_ext cython
%matplotlib inline
import sys
sys.path.append('/home/bram/ANTS')
sys.path.append('/home/bram/ANTS/entropy')
for p in sys.path:
    print(p)


/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages/IPython/extensions
/home/bram/.ipython
/home/bram/ANTS
/home/bram/ANTS/entropy


In [43]:
%%cython
# distutils: language = c++
from libcpp.vector cimport vector
from cythonic.plugins.db_controller cimport db_controller
from cythonic.plugins.db_path import db_path
from cythonic.plugins.queries import get_settings, get_steps
from cythonic.plugins.positions cimport point
from cythonic.core.domain cimport Domain
cimport cython

from libc.math cimport M_PI, sin as csin, cos as ccos

import numpy as np

db = db_controller(db_path(), 'stigmergy.db')
def extract_settings(rows, headers):
    return {headers[i]:rows[0][i] for i in range(len(headers))}

cdef str qry = "SELECT * FROM sim"

cdef double rotate_x(double l, double theta):
    "rotate by theta in radians "
    return ccos(theta)*l

cdef double rotate_y(double l, double theta):
    return csin(theta)*l

cdef double deg2rad(double theta ):
    return theta*M_PI/180

# cdef point text2point(str txt):
#     cdef list lst = eval(txt)
#     cdef point result = {'x':lst[0], 'y':lst[1]}
#     return result

# data, headers = db.return_all(qry)
# print(extract_settings(headers, data))


@cython.boundscheck(True)
@cython.wraparound(True)
cdef class SimPlayer:
    cdef:
        readonly object db, steps
        readonly unsigned int n_agents, count_active
        readonly unsigned int id
        double evap_rate
        Domain domain
        
        readonly double ant_size, sens_offset
        double[:,::] positions
        double[:] headings
        double[:,::] lefts
        double[:,::] rights
        
        readonly unsigned int[:] steplist
        readonly double[:] entropy
        readonly unsigned int[:] foodcount
    
    def __init__(self,sim_id, db_path, db_name):
        self.db = db_controller(db_path, db_name)
        self.id = sim_id
        d,h = db.return_all(get_settings(sim_id, 'sim_settings'))
        sim_settings = extract_settings(*db.return_all(get_settings(sim_id,'sim_settings')))
        ant_settings = extract_settings(*db.return_all(get_settings(sim_id,'ant_settings')))
        dom_settings = extract_settings(*db.return_all(get_settings(sim_id,'domain_settings')))
        gauss_settings = extract_settings(*db.return_all(get_settings(sim_id,'gauss_settings')))

        self.evap_rate = sim_settings['evap_rate']
        self.n_agents = sim_settings['n_agents']
        self.ant_size = ant_settings['l']
        self.sens_offset = ant_settings['sens_offset']
        self.positions = np.zeros([self.n_agents,2],dtype = np.float_)
        self.headings = np.zeros(self.n_agents, dtype =np.float_)
        self.lefts = np.zeros([self.n_agents,2],dtype = np.float_)
        self.rights = np.zeros([self.n_agents,2],dtype = np.float_)
        
        
        " initialize the objects "
        self.init_domain(dom_settings)
        self.init_gaussian(gauss_settings)
        self.count_active = 0
        
    def get_results(self,):
        results = extract_settings(*db.return_all(get_settings(self.id,'results')))
        " check if step_vec, scorecard and entropy_vec are not NULL, then store as object attribute "
        if results['step_vec']:
            self.steplist = np.asarray(eval(results['step_vec']),dtype=np.uint32)
            if not results['scorecard']:
                " no results in query "
                self.foodcount = np.zeros(self.steplist.shape[0], dtype = np.uint32)
            else:
                self.foodcount = np.asarray(eval(results['scorecard']),dtype = np.uint32)
            if not results['entropy_vec']:
                " no results in query "
                self.entropy = np.zeros(self.steplist.shape[0], dtype = np.float_)
            else:
                self.entropy = np.asarray(eval(results['entropy_vec']),dtype = np.float_)
        
    def init_domain(self,dict domain_dict):
        " extract the query into useable initialization parameters "
        del domain_dict['sim_id'] # remove sim_id from dict
        cdef dict dom_dict = {}
        for key, value in domain_dict.items():
            if type(value) is str:
                dom_dict[key] = eval(value)
            else:
                dom_dict[key] = value
        self.domain = Domain(**dom_dict)
        
    def init_gaussian(self, gauss_dict):
        del gauss_dict['sim_id'] # remove sim_id from dict
        self.domain.init_gaussian(gauss_dict['covariance'], gauss_dict['significancy'])
        
    
    def get_steps(self ):
        " get dataframe from qry "
        qry = get_steps(self.id)
        self.steps = db.get_df(qry)
        
    cdef step(self, unsigned int stepnr):
        " do a step "
        cdef double x,y, theta, q
        cdef point pos
        cdef unsigned int i = 0 # iterator for positions, headings, lefts and rights
        for row in self.steps[self.steps['STEP_NR']== stepnr].itertuples(index = False):
            pos.x = row.X
            pos.y = row.Y
            theta = row.THETA
            q = row.Q
            self.positions[i,0] = pos.x
            self.positions[i,1] = pos.y
            self.headings[i] = theta
            self.lefts[i,0] = pos.x+rotate_x(l = self.ant_size,theta = deg2rad(theta+self.sens_offset))
            self.lefts[i,1] = pos.y+rotate_y(l = self.ant_size,theta = deg2rad(theta+self.sens_offset))
            self.rights[i,0] = pos.x+rotate_x(l = self.ant_size,theta = deg2rad(theta-self.sens_offset))
            self.rights[i,1] = pos.y+rotate_y(l = self.ant_size,theta = deg2rad(theta-self.sens_offset))
            self.domain.add_pheromone(p = &pos, Q = &q)
            
            i+=1
        self.domain.evaporate(tau = &self.evap_rate)           
        self.count_active = i
        
    @property
    def pos_x(self):
        return np.asarray(self.positions[:self.count_active,0])
    @property
    def pos_y(self):
        return np.asarray(self.positions[:self.count_active,1])
    @property
    def left_x(self):
        return np.asarray(self.lefts[:self.count_active,0])
    @property
    def left_y(self):
        return np.asarray(self.lefts[:self.count_active,1])
    @property
    def right_x(self):
        return np.asarray(self.rights[:self.count_active,0])
    @property
    def right_y(self):
        return np.asarray(self.rights[:self.count_active,1])
    @property
    def heading(self):
        return np.asarray(self.headings[:self.count_active])
    @property
    def H(self ):
        return np.asarray(self.entropy)[np.asarray(self.steplist)<=<unsigned int>100]
        
    
        
        
from pandas import read_sql, DataFrame as DF
print(get_settings('5',table='sim_settings'))
cdef SimPlayer player = SimPlayer(84,db_path(), 'stigmergy.db')
# player.get_steps()
# player.get_results()
# Y = np.asarray(player.entropy)[np.asarray(player.steplist)<=100]
# T = np.asarray(player.steplist)
# print(player.H)



# for row in player.steps.itertuples(index=True, ):
#     print(row.X, row.Y, row.THETA)
# doorgaan = True
# i = 10
# import time
# tic = time.time()
# player.step(3)
# toc = time.time()
# print(1000*(toc-tic))
# print(player.pos_x)

print(np.vstack([[rotate_x(l=player.domain.nest_radius, theta = theta),
                 rotate_y(l=player.domain.nest_radius, theta = theta)]
                 for theta in np.arange(0,2*M_PI,M_PI/10)]))


SELECT * FROM sim_settings WHERE sim_settings.sim_id = 5
[[ 5.00000000e+01  0.00000000e+00]
 [ 4.75528258e+01  1.54508497e+01]
 [ 4.04508497e+01  2.93892626e+01]
 [ 2.93892626e+01  4.04508497e+01]
 [ 1.54508497e+01  4.75528258e+01]
 [ 3.06161700e-15  5.00000000e+01]
 [-1.54508497e+01  4.75528258e+01]
 [-2.93892626e+01  4.04508497e+01]
 [-4.04508497e+01  2.93892626e+01]
 [-4.75528258e+01  1.54508497e+01]
 [-5.00000000e+01  6.12323400e-15]
 [-4.75528258e+01 -1.54508497e+01]
 [-4.04508497e+01 -2.93892626e+01]
 [-2.93892626e+01 -4.04508497e+01]
 [-1.54508497e+01 -4.75528258e+01]
 [-9.18485099e-15 -5.00000000e+01]
 [ 1.54508497e+01 -4.75528258e+01]
 [ 2.93892626e+01 -4.04508497e+01]
 [ 4.04508497e+01 -2.93892626e+01]
 [ 4.75528258e+01 -1.54508497e+01]]


In [42]:
%%cython -a
from libc.math cimport log2 as clog2
import numpy as np
cdef double[:,::] M = np.random.rand(20,20)
cdef unsigned int i,j
(i,j) = M.shape[:2]

cdef double csum(double[:,::] M):
    cdef unsigned int i,j, I,J
    (I,J) = M.shape[:2]
    cdef double s=0.
    for i in range(I):
        for j in range(J):
            s+=M[i,j]
    return s

cdef double H(double[:,::] M):
    cdef unsigned int i,j,I,J
    (I,J) = M.shape[:2]
    cdef double h = 0.
    cdef double T = csum(M)
    cdef str err_txt = 'log of nearnegative number is ill-defined'
    if T <= 0.0:
        raise ValueError('Sum of pheromone map <= 0')
    for i in range(I):
        for j in range(J):
            if M[i,j]<= 1e-10:
                raise ValueError(err_txt)
            h-= M[i,j]/T*clog2(M[i,j]/T)
    return h

cdef double ent = H(M)
print(ent)

cdef unsigned int[:] rw = np.arange(0,10,1,dtype=np.uint32)
print(np.asarray(rw).tolist())

from pandas import DataFrame as DF

df = DF({'x':range(10)})
print(df['x'].values)
cdef int[:] x = np.asarray(df['x'].values, dtype=np.int32)

8.341745451487753
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0 1 2 3 4 5 6 7 8 9]


In [13]:
%%cython -a
import numpy as np
t = np.arange(10)

cdef unsigned int steps = 10

cdef unsigned int[:] k_vec = np.arange(0,steps+1, <unsigned int>int(np.ceil(steps/100)), dtype = np.uint32)
print(np.asarray(k_vec).tolist())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
